# Monte Carlo off-policy control with weighted importance sampling

In [ ]:
import numpy as np
import gym

In [ ]:
env = gym.make('FrozenLake8x8-v0')

In [ ]:
#define behavior policy, mu, and target policy, pi
def mu(state):
    return np.random.randint(0,4)

def pi(Q,state):
    return Q[observation,:].argmax()

In [ ]:
def mc_off_policy_weighted(n_episodes,env,gamma):
    
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    #initialize action-value function
    Q = np.zeros((n_states,n_actions))
    
    #initialize counter (counting across episodes)
    C = np.zeros((n_states,n_actions))
    
    for episode in range(n_episodes):
        observation = env.reset()
        state_action_reward_tuples = []
        done = False
        
        #generate episode using mu
        while not done:
            action = mu(observation)
            old_observation = observation
            observation, reward, done, info = env.step(action)
            state_action_reward_tuples.append((observation,action,reward))
            
        G = 0
        W = 1
        
        for state,action,reward in state_action_reward_tuples[::-1]:
            G = reward + gamma * G
            C[state][action] += W
            Q[state][action] += (W/C[state][action])*(G-Q[state][action])
            if action != pi(Q,state):
                break
            W = W * 1/0.25
            
    return Q

In [ ]:
n_states = env.observation_space.n
n_actions = env.action_space.n
    
    #initialize action-value function
Q = np.zeros((n_states,n_actions))
    
    #initialize counter (counting across episodes)
C = np.zeros((n_states,n_actions))
    

for episode in range(20000):
    observation = env.reset()
    state_action_reward_tuples = []
    done = False
    
    #generate episode using mu
    while not done:
        action = mu(observation)
        old_observation = observation
        observation, reward, done, info = env.step(action)
        state_action_reward_tuples.append((observation,action,reward))
        
    G = 0
    W = 1
    
    for state,action,reward in state_action_reward_tuples[::-1]:
        G = reward + gamma * G
        C[state,action] += W
        Q[state,action] += (W/C[state,action])*(G-Q[state,action])
        if action != pi(Q,state):
            break
        W = W * 1/0.25

In [ ]:
gamma = 0.9
Q = mc_off_policy_weighted(20000,env,gamma)

In [ ]:
Q